# Analyzing and Visualizing Data from Foxglove Data Platform

*NOTE: Corresponding [blog post here](https://foxglove.dev/blog).*

**[Foxglove Data Platform](https://foxglove.dev/data-platform) is a scalable platform for organizing and managing your team's robotics data.** You can log in to [its web console interface](https://console.foxglove.dev) to upload data, tag events of interest, and query data for a given robot and time range, even if that data spans multiple recording sessions.

In this notebook, **we'll demonstrate how to retrieve messages from Data Platform and process them for insightful visualizations.** We'll be using self-driving car data from the [nuScenes dataset](https://www.nuscenes.org/nuscenes), and writing Python code to visualize its route, IMU acceleration, and perceived objects.

In [ ]:
# We need to install some dependencies

!pip install --index-url https://rospypi.github.io/simple rosbag
!pip install mcap-ros1-support foxglove-data-platform pandasql

## 2. Create a Foxglove Data Platform client

Next, we'll create an instance of our Foxglove Data Platform client that we'll use to retrieve messages.

In [ ]:
from foxglove_data_platform.client import Client

# Read-only public key for demonstration purposes
client = Client(token="fox_sk_CFlhbEgxMQbHBQGRCNE7c3CBaOxtrpk5")

## 3. View available data coverage

*We'll be using the Python [`datetime`](https://docs.python.org/3/library/datetime.html) and [`pandas`](https://pandas.pydata.org/) modules for this step.*

Lastly, we'll view the available data coverage in our Foxglove Data Platform account, to determine the devices and time ranges we need to query for the data we want to visualize.

Let's limit the scope of our visualization to 2018 data:

In [ ]:
from datetime import datetime
import pandas as pd

coverage = client.get_coverage(start=datetime(2018, 1, 1), end=datetime(2019,1,1))
coverage = sorted(coverage, key=lambda c: c['start'])
pd.DataFrame(coverage).head()

In [ ]:
# Now let's download some GPS messages.

gps_messages = [
    (message.latitude, message.longitude)
    for topic, record, message in client.get_messages(
        device_id=coverage[1]["device_id"],
        start=coverage[1]["start"],
        end=coverage[1]["end"],
        topics=["/gps"],
    )
]
pd.DataFrame(gps_messages, columns=["lat", "lon"]).head()

Each row in the output above corresponds to every discrete recording available between January 1, 2018 and January 1, 2019.

We see that there is only one device (i.e. robot) with 2018 data – we'll be using its ID (dev_vsXaJAH6X6oWDJrw) when querying its data in the next steps.

# Mapping the route

To start exploring our self-driving data, let's see the route that our car took by plotting its GPS coordinates on a map.

Now that we know the data available to us, we can fetch it using our Data Platform client's `get_messages` API call. 

Let's focus on one recording – specifically the ride that happened on August 1, 2018 (`coverage[1]`). We'll use a Python list comprehension to convert our messages into a list of tuples that we can easily insert into a `pandas` dataframe.

In [ ]:
gps_messages = [
    (message.latitude, message.longitude)
    for topic, record, message in client.get_messages(
        device_id=coverage[1]["device_id"],
        start=coverage[1]["start"],
        end=coverage[1]["end"],
        topics=["/gps"],
    )
]
pd.DataFrame(gps_messages, columns=["lat", "lon"]).head()

We can use this same list of tuples to plot a list of locations on a [`folium`](https://python-visualization.github.io/folium/) map.

In [ ]:
import folium

figure = folium.Figure(width=640, height=480)
map = folium.Map(location=gps_messages[0], zoom_start=200, width="100%")
folium.PolyLine(
    locations=gps_messages,
    weight=10,
    color="purple",
).add_to(map)
map.add_to(figure)

We can see that on August 1, 2018, our self-driving car navigated a stretch of Congress Street in Boston.

# Plotting IMU acceleration

For our first visualization, we focused on just one recorded drive. For this next one, let's fetch messages across multiple recording sessions to plot our robot's acceleration across all 2018 drives.

We can take advantage of Data Platform's ability to fetch messages across multiple recording sessions by specifying the time range we want in our `get_messages` call. Once again, we'll use a list comprehension to convert our message objects into dictionaries that we can pull into a `pandas` dataframe and plot.

In [ ]:
imu_messages = [
    {
        "time": pd.Timestamp(message.header.stamp.to_nsec(), unit="ns").isoformat(),
        "accel_x": message.linear_acceleration.x,
        "accel_y": message.linear_acceleration.y,
    }
    for topic, record, message in client.get_messages(
        device_id=coverage[0]["device_id"],
        start=coverage[0]["start"],
        end=coverage[-1]["end"],
        topics=["/imu"],
    )
]
pd.DataFrame(imu_messages).plot(x="time", figsize=(10, 6), rot=45)

From the output above, we can see how our robot's x and y acceleration fluctuated throughout its 2018 drives.

# Classifying perceived object markers

Finally, let's classify the perceived object markers that our self-driving car published while on the road.

Let's focus on one drive again this time – `coverage[1]` – and query its `/markers/annotations` topic messages.

In [ ]:
marker_messages = client.get_messages(
    device_id=coverage[1]["device_id"],
    start=coverage[1]["start"],
    end=coverage[1]["end"],
    topics=["/markers/annotations"],
)

We'll use the markers' colors as the keys in a `color_to_classname` lookup dictionary. 

Use [`matplotlib`](https://matplotlib.org/) to convert numbers between 0 to 1 into hex color codes, and [`pandasql`](https://pypi.org/project/pandasql/) to count and group the markers by type.

In [ ]:
import matplotlib as mpl
from pandasql import sqldf

color_to_classname = {
    "#000000": "noise",
    "#468250": "animal",
    "#0000e6": "human.pedestrian.adult",
    "#87ceeb": "human.pedestrian.child",
    "#f08080": "human.pedestrian.construction_worker",
    "#db7093": "human.pedestrian.personal_mobility",
    "#000080": "human.pedestrian.police_officer",
    "#f08080": "human.pedestrian.stroller",
    "#8a2be2": "human.pedestrian.wheelchair",
    "#708090": "movable_object.barrier",
    "#d2691e": "movable_object.debris",
    "#696969": "movable_object.pushable_pullable",
    "#2f4f4f": "movable_object.trafficcone",
    "#bc8f8f": "static_object.bicycle_rack",
    "#dc143c": "vehicle.bicycle",
    "#ff7f50": "vehicle.bus.bendy",
    "#ff4500": "vehicle.bus.rigid",
    "#ff9e00": "vehicle.car",
    "#e99646": "vehicle.construction",
    "#ffd700": "vehicle.emergency.ambulance",
    "#ffd700": "vehicle.emergency.police",
    "#ff3d63": "vehicle.motorcycle",
    "#ff8c00": "vehicle.trailer",
    "#ff6347": "vehicle.truck",
    "#00cfbf": "flat.driveable_surface",
    "#af004b": "flat.other",
    "#4b004b": "flat.sidewalk",
    "#70b43c": "flat.terrain",
    "#deb887": "static.manmade",
    "#ffe4c4": "static.other",
    "#00af00": "static.vegetation",
    "#fff0f5": "vehicle.ego",
}

flattened_markers = []
for topic, record, message in marker_messages:
    for marker in message.markers:
        color = mpl.colors.to_hex([marker.color.r, marker.color.g, marker.color.b])
        class_name = color_to_classname[color]
        flattened_markers.append((marker.text, class_name))
annotations = pd.DataFrame(flattened_markers, columns=["annotation_id", "class_name"])

pysqldf = lambda q: sqldf(q, globals())
pysqldf(
    "SELECT class_name,COUNT(*) as count FROM annotations GROUP BY class_name ORDER BY count DESC"
)

From the output above, we can see how many examples of each perceived object our self-driving car encountered (943 cars, 879 adult pedestrians, etc.).

# Conclusion

We hope this demo illustrated some of the many ways [Foxglove Data Platform](https://foxglove.dev/data-platform) can help you explore, manipulate, and visualize your robotics data – even outside the [Foxglove Studio app](https://foxglove.dev/studio).

Join the Foxglove [Slack community](https://foxglove.dev/slack) and follow [our blog](https://foxglove.dev/blog) for more ideas on how to integrate Data Platform into your robotics development workflows.